In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
import pandas as pd
tf.config.set_soft_device_placement(False)
tf.debugging.set_log_device_placement(True)
import numpy as np
from ampligraph.datasets import load_fb15k_237, load_yago3_10
from ampligraph.evaluation.protocol import create_mappings, to_idx

from ampligraph.evaluation.metrics import mrr_score, hits_at_n_score, mr_score
import time
print(tf.__version__)
assert(tf.__version__.startswith('2.3'))

from ampligraph.datasets import load_fb15k_237, load_fb13, load_fb15k, load_wn11, load_wn18, load_wn18rr, load_yago3_10
from ampligraph.latent_features import ScoringBasedEmbeddingModel

2.3.1


In [2]:
#loss function
def nll(scores_pos, scores_neg, eta):
    scores_neg_reshaped = tf.reshape(scores_neg, [eta, tf.shape(scores_pos)[0]])
    neg_exp = tf.exp(scores_neg_reshaped)
    pos_exp = tf.exp(scores_pos)
    softmax_score = pos_exp / (tf.reduce_sum(neg_exp, axis=0) + pos_exp)

    loss = -tf.reduce_sum(tf.math.log(softmax_score))
    return loss

# Train/eval without partition

In [3]:
optim = tf.optimizers.Adagrad(learning_rate=0.1)
# optim = 'adam'

# loss = nll
# loss = 'self_adversarial'
from ampligraph.latent_features.loss_functions import SelfAdversarialLoss
loss = SelfAdversarialLoss({'margin': 0.1, 'alpha': 5})

model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300,
                                     scoring_type='TransE')



model.compile(optimizer=optim, loss=loss)

from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')

model.fit('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
             batch_size=10000,
             epochs=10,
             validation_freq=5,
             validation_batch_size=100,
             validation_data = '/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
         callbacks=[checkpoint])




Epoch 1/10
     29/Unknown - 1s 48ms/step - loss: 13395.2031
Epoch 00001: loss improved from inf to 13395.20312, saving model to ./chkpt1
29/29 [==============================] - 1s 49ms/step - loss: 13395.2031
Epoch 2/10
28/29 [===========================>..] - ETA: 0s - loss: 12971.0254
Epoch 00002: loss improved from 13395.20312 to 12786.13477, saving model to ./chkpt1
29/29 [==============================] - 1s 38ms/step - loss: 12786.1348
Epoch 3/10
28/29 [===========================>..] - ETA: 0s - loss: 12628.9600
Epoch 00003: loss improved from 12786.13477 to 12509.62305, saving model to ./chkpt1
29/29 [==============================] - 1s 38ms/step - loss: 12509.6230
Epoch 4/10
28/29 [===========================>..] - ETA: 0s - loss: 12431.5859
Epoch 00004: loss improved from 12509.62305 to 12343.49023, saving model to ./chkpt1
29/29 [==============================] - 1s 38ms/step - loss: 12343.4902
Epoch 5/10
28/29 [===========================>..] - ETA: 0s - loss: 12299.8691

In [4]:

start = time.time()
ranks = model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 94s 456ms/step
Time taken: 101.44613218307495


(329.31461004012135,
 0.20401738341398626,
 0.128265975144339,
 0.3547313827184656)

## Load Checkpoint and evaluate

In [5]:
start = time.time()
loaded_model = ScoringBasedEmbeddingModel(eta=50, 
                                     k=300, 
                                     scoring_type='TransE')
loaded_model.load_weights('./chkpt1')
ranks = loaded_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=100,
                       corrupt_side='s,o',
         use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                  'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                  'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
206/206 [==============================] - 91s 440ms/step
Time taken: 98.82936143875122


(329.31461004012135,
 0.20401738341398626,
 0.128265975144339,
 0.3547313827184656)

# Training/eval with partition

In [6]:
from ampligraph.datasets import DummyBackend, SQLiteAdapter
from ampligraph.datasets import GraphDataLoader
from ampligraph.datasets.graph_partitioner import PARTITION_ALGO_REGISTRY
dataset_loader = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt', 
                                 backend=SQLiteAdapter,
                                    batch_size=1000, 
                                    dataset_type='train', 
                                    epochs=10,
                                    use_indexer=True)


partitioner = PARTITION_ALGO_REGISTRY.get('Bucket')(dataset_loader, k=3)

_split: memory before: 0.0Bytes, after: 13.189MB, consumed: 13.189MB; exec time: 213.34s


In [7]:
optim = tf.optimizers.Adam(learning_rate=0.1, amsgrad=True)

partitioned_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')

#model.compile(optimizer='rmsprop', loss=nll)
partitioned_model.compile(optimizer=optim, loss=nll)
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./chkpt1', monitor='loss', verbose=1, save_best_only=True, mode='min')
start = time.time()
partitioned_model.fit(partitioner,
                     batch_size=1000, use_partitioning=True,
                     epochs=10, callbacks=[checkpoint])
print((time.time()-start))

Epoch 1/10
    276/Unknown - 10s 35ms/step - loss: 1404.1381
Epoch 00001: loss improved from inf to 1399.37598, saving model to ./chkpt1
277/277 [==============================] - 10s 38ms/step - loss: 1399.3760
Epoch 2/10
276/277 [============================>.] - ETA: 0s - loss: 1143.7428
Epoch 00002: loss improved from 1399.37598 to 1141.77954, saving model to ./chkpt1
277/277 [==============================] - 10s 35ms/step - loss: 1141.7795
Epoch 3/10
276/277 [============================>.] - ETA: 0s - loss: 940.3823
Epoch 00003: loss improved from 1141.77954 to 939.28552, saving model to ./chkpt1
277/277 [==============================] - 10s 36ms/step - loss: 939.2855
Epoch 4/10
276/277 [============================>.] - ETA: 0s - loss: 821.0338
Epoch 00004: loss improved from 939.28552 to 820.29999, saving model to ./chkpt1
277/277 [==============================] - 10s 36ms/step - loss: 820.3000
Epoch 5/10
276/277 [============================>.] - ETA: 0s - loss: 742.7835
Ep

In [8]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=partitioned_model.data_handler.get_mapper())

start = time.time()
ranks = partitioned_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
52/52 [==============================] - 126s 2s/step
Time taken: 128.60456943511963


(849.2038725490196, 0.06923169819362532, 0.0, 0.19223039215686274)

## Save and Load checkpoint

In [9]:
partitioned_model.save_weights('./chkpt1_partition')

In [10]:
loaded_part_model = ScoringBasedEmbeddingModel(eta=5, 
                                     k=300, 
                                     scoring_type='TransE')

loaded_part_model.load_weights('./chkpt1_partition')

In [11]:

dataset_loader_test = GraphDataLoader('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                                        backend=SQLiteAdapter,
                                        batch_size=400, 
                                        dataset_type='test', 
                                        epochs=1,
                                        use_indexer=loaded_part_model.data_indexer)

start = time.time()
ranks = loaded_part_model.evaluate(dataset_loader_test, 
                                   batch_size=400)
end = time.time()
print('Time taken:', end-start)

mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!
52/52 [==============================] - 130s 2s/step
Time taken: 132.51785135269165


(849.2038725490196, 0.06923169819362532, 0.0, 0.19223039215686274)

## Filtered evaluation

In [12]:
start = time.time()
ranks = partitioned_model.evaluate('/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt', 
                       batch_size=200,
                       corrupt_side='s,o',
                        use_filter={'train':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/train.txt',
                              'valid':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/valid.txt',
                              'test':'/home/spai/code/ampligraph_projects/dataset/fb15k-237/test.txt'})
end = time.time()
print('Time taken:', end-start)
mr_score(ranks), mrr_score(ranks), hits_at_n_score(ranks, 1), hits_at_n_score(ranks, 10)


28 triples containing invalid keys skipped!

9 triples containing invalid keys skipped!

28 triples containing invalid keys skipped!
104/104 [==============================] - 716s 7s/step
Time taken: 775.3156113624573


(712.6684607104413,
 0.13523140769564457,
 0.08178393189157451,
 0.24192680301399355)